## 🎯 Training Loop
We train for 10 epochs and keep the best model (highest validation accuracy).


## 🔧 Load Pretrained Model (ResNet18)
We use ResNet18 from torchvision, replace the final layer with 2 output neurons (Normal / Pneumonia).


In [13]:
import os
import copy
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from tqdm import tqdm

# Device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
data_dir = "../data/chest_xray"  # make sure you unzipped dataset here

#2. Data Preparation
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# print(os.getcwd())
train_data = datasets.ImageFolder(os.path.join(data_dir, "train"), transform=train_transforms)
val_data   = datasets.ImageFolder(os.path.join(data_dir, "val"), transform=test_transforms)
test_data  = datasets.ImageFolder(os.path.join(data_dir, "test"), transform=test_transforms)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_data, batch_size=32, shuffle=False)
test_loader  = DataLoader(test_data, batch_size=32, shuffle=False)

print("Classes:", train_data.classes)
print("Train samples:", len(train_data))
print("Val samples:", len(val_data))
print("Test samples:", len(test_data))

# Load pretrained ResNet18
model = models.resnet18(pretrained=True)

# Replace final layer for 2 classes
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)

model = model.to(device)

# Loss & optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

def train_model(model, criterion, optimizer, num_epochs=10):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        print("-" * 30)

        for phase in ['train', 'val']:
            model.train() if phase == 'train' else model.eval()
            dataloader = train_loader if phase == 'train' else val_loader

            running_loss, running_corrects = 0.0, 0
            loop = tqdm(dataloader, desc=f"{phase} phase", leave=False)

            for inputs, labels in loop:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                loop.set_postfix(loss=loss.item())

            epoch_loss = running_loss / len(dataloader.dataset)
            epoch_acc = running_corrects.double() / len(dataloader.dataset)

            print(f"{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

            # Track best validation accuracy
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

    print(f"\n✅ Best val Acc: {best_acc:.4f}")
    model.load_state_dict(best_model_wts)
    return model
model = train_model(model, criterion, optimizer, num_epochs=10)

# Save model
os.makedirs("src/models", exist_ok=True)
torch.save(model.state_dict(), "src/models/pneumonia_resnet18.pt")
print("Model saved ✅")



Using device: cuda
Classes: ['NORMAL', 'PNEUMONIA']
Train samples: 5216
Val samples: 16
Test samples: 624


c:\Users\abdel\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\abdel\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\abdel/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:05<00:00, 8.38MB/s]



Epoch 1/10
------------------------------


...

val Loss: 0.1049 Acc: 1.0000

Epoch 9/10
------------------------------


train Loss: 0.0179 Acc: 0.9921


val Loss: 0.0425 Acc: 1.0000

Epoch 10/10
------------------------------


train Loss: 0.0194 Acc: 0.9927


val Loss: 0.1137 Acc: 0.9375

✅ Best val Acc: 1.0000
Model saved ✅


# 🧠 Baseline Model Training – Pneumonia Detection

In this notebook, we will:
- Use a pre-trained CNN (ResNet18) with transfer learning.
- Train it for binary classification (Normal vs Pneumonia).
- Save the best model checkpoint.
